In [334]:
import os
import numpy as np
import scipy.misc
from stylize import stylize
import math
from argparse import ArgumentParser

import vgg
import tensorflow as tf
import numpy as np
from sys import stderr

In [335]:
# default arguments
VGG_PATH = 'imagenet-vgg-verydeep-19.mat'
CONTENT_LAYER = 'relu4_2'
VERTICAL_SHAPE = (682, 512, 3)
HORIZONTAL_SHAPE = (384, 512, 3)

In [336]:
def imread(path):
    return scipy.misc.imread(path)

def imresize(x, shape):
    return scipy.misc.imresize(x, shape)

In [337]:
def features(library_type, options):
    library = options.library_vertical
    target_shape = VERTICAL_SHAPE
    
    if library_type == 'horizontal':
        library = options.library_horizontal
        target_shape = HORIZONTAL_SHAPE
        
    style_paths = [os.path.join(library, f) for f in os.listdir(library) if f.endswith('.jpg') or f.endswith('.png')]
    style_features = [{} for _ in style_paths]
    # compute style features in feedforward mode
    for i in range(len(style_paths)):
        style = imread(style_paths[i])
        style = scipy.misc.imresize(style, target_shape)
        style_shape = (1,) + style.shape
        g = tf.Graph()
        with g.as_default(), g.device('/cpu:0'), tf.Session() as sess:
            image = tf.placeholder('float', shape=style_shape)
            net, _ = vgg.net(network, image)
            style_pre = np.array([vgg.preprocess(style, mean_pixel)])
            style_features[i][CONTENT_LAYER] = net[CONTENT_LAYER].eval(feed_dict={image: style_pre})
    return style_paths, style_features

In [348]:
class Args(dict):
    pass

options = Args()
options.content = 'examples/google2.jpg'
options.library_vertical = 'library/vertical/'
options.library_horizontal = 'library/horizontal/'
options.network = VGG_PATH

In [349]:
content = imread(options.content)
network = options.network

content_shape = content.shape
target_shape = VERTICAL_SHAPE
if (content_shape[0] < content_shape[1]):
    target_shape = HORIZONTAL_SHAPE
content = imresize(content, target_shape)

print(target_shape)




(384, 512, 3)


In [350]:
shape = (1,) + content.shape
content_features = {}

g = tf.Graph()
# compute content features in feedforward mode
with g.as_default(), g.device('/cpu:0'), tf.Session() as sess:
    image = tf.placeholder('float', shape=shape)
    net, mean_pixel = vgg.net(network, image)
    content_pre = np.array([vgg.preprocess(content, mean_pixel)])
    content_features[CONTENT_LAYER] = net[CONTENT_LAYER].eval(
            feed_dict={image: content_pre})

In [331]:
style_paths_vertical, style_features_vertical = features('vertical', options)
style_paths_horizontal, style_features_horizontal = features('horizontal', options)

In [351]:
distance = []

style_features = style_features_vertical
style_paths = style_paths_vertical

if target_shape[0] < target_shape[1]:
    style_features = style_features_horizontal 
    style_paths = style_paths_horizontal

for i in range(len(style_features)):
    g = tf.Graph()
    with g.as_default(), g.device('/cpu:0'), tf.Session() as sess:
        ###distance between input content and style content
        content_loss = tf.nn.l2_loss(
            style_features[i][CONTENT_LAYER] - content_features[CONTENT_LAYER]) / content_features[CONTENT_LAYER].size
        distance.append(content_loss.eval())

In [352]:
indices = [i[0] for i in sorted(enumerate(distance), key=lambda x:x[1])]
images = []
for i in range(len(indices)):
    images.append(style_paths[indices[i]])
print(distance)
images


[254612.39, 269604.44, 268701.34, 313079.84, 298431.16, 305856.78, 276659.94, 307454.66, 329315.09, 312541.75, 286039.91, 291221.53, 366523.03, 280096.72]


['library/horizontal/1905.jpg',
 'library/horizontal/maria-petipa.jpg',
 'library/horizontal/ivan-ilyin.jpg',
 'library/horizontal/self-portrait-1908.jpg',
 'library/horizontal/self-portrait.jpg!Large.jpg',
 'library/horizontal/self-portrait-in-the-garden.jpg!Large.jpg',
 'library/horizontal/self-portrait-vistula-river-behind.jpg!Large.jpg',
 'library/horizontal/portrait-of-zacherie-zacharian-1886.jpg!Large.jpg',
 'library/horizontal/self-portrait-1905.jpg!Large(1).jpg',
 'library/horizontal/self-portrait-1909.jpg',
 'library/horizontal/self-portrait-1953.jpg!Large.jpg',
 'library/horizontal/portrait-of-simone-fayet-1907.jpg!Large.jpg',
 'library/horizontal/self-portrait-1946.jpg',
 'library/horizontal/self-portrait-with-a-sunflower-1632.jpg!Large.jpg']

In [281]:
style_paths[0]

SyntaxError: invalid syntax (<ipython-input-281-f34fb8720f3d>, line 1)